# Challenge

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
file = "C:/Users/guzma/OneDrive/Documents/TEC/S6/MA2007/Reto/noaa_ai_workshop/IntroNotebooks/uke_eggedal_data_challenge.pkl"
df = pd.read_pickle(file)

## Data Cleaning

In [18]:
# print(df.keys())
df1 = pd.DataFrame(df['flow'])
df1.columns = ['flow']
df2 = pd.DataFrame(df['obs_tas'])
df2.columns = ['tas_0', 'tas_1', 'tas_2', 'tas_3', 'tas_4', 'tas_5', 'tas_6', 'tas_7', 'tas_8']
df3 = pd.DataFrame(df['flow_dates'])
df4 = pd.DataFrame(df['obs_lon_lat'])
df5 = pd.DataFrame(df['obs_pr'])
df5.columns = ['pr_0', 'pr_1', 'pr_2', 'pr_3', 'pr_4', 'pr_5', 'pr_6', 'pr_7', 'pr_8']
df6 = pd.DataFrame(df['obs_dates'])